## Fundamentals of Data Science: Final Presentation

<center><img src="TeamPicture.png" alt="drawing" width="900"/></center>

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
import seaborn as sns
from astropy.time import Time
from collections import Counter

sns.set_context("talk")
sns.set_style("white")
sns.set_palette("Dark2")

import sys
import warnings

if not sys.warnoptions:
    warnings.simplefilter("ignore")
    
import logging
logger = logging.getLogger("pymc3")
logger.propagate = False

*Charmi*
## Supernovae
<center><img src="SN1994D.jpg" alt="drawing" width="500"/></center>

SN are incredibly complex events, and all we have to study them are counts on a detector. To understand these explosions and the stars that caused them, we have to understand the detailed evolution of an SN's energy output. 

*Charmi*
# Objects in this project

* 2020oi - Type Ic
* LSQ12gdj - Type Ia
* 1987A - Type II
* 2011fe - Type Ia

Why did we choose these? We wanted a spread in data quality and types of supernovae. 1987A and 2011fe are well studied and good benchmarks. 2020oi is a newly discovered event that Alex is working on in real life*. LSQ12gdj was chosen because it is the main focus of Scalzo et. al. 2014, where the authors similarly obtain a bolometric light curve.

*real life is not class.

*Charmi*

## The Bolometric Problem
SN emit in nearly all wavelengths, but we aren't able to take a spectrum continuously throughout the entire event. 


* Dust extinction masks blue/UV emission, contributes IR emission

* We miss some flux before event discovery

* Bands are discretized - e.g. ugriz, UBVRI - so photometry tells a small part of the story. 

We need some way to estimate the amount of energy these SN emitted (or continue to emit) in all wavelengths, while only measuring a few. 

*Cyrus*
## Gaussian Processes (GPs) to the rescue! 
(With hierarchical models to boot)

A **Gaussian Process** is a stochastic process consisting of random variables whose every finite linear combination is normally distributed. If we choose our random variables (and our linear combinations) intelligently, we can use them to fit data. 

*Cyrus*
## Part I: Creating a Mean Model
We opted for a spline of two quadratic functions. We attempted two different models: 
* Spline at maximum (peak brightness in $r$-band)
* Spline during decline (located by visual inspection)

<center><img src="plots/MeanModels.png" alt="drawing" width="900"/></center>

*Cyrus*
## Part II: With George, we can condition on the data (with a squared exponential kernel) for a better fitting model. 

<center><img src="plots/George.png" alt="drawing" width="900"/></center>

*Melanie*
## Part III: Gaussian Processes Implemented in PyMC3
So that we'll be able to use a hierarchical model!

One main difference between the GPs in george and PyMC3 is that george let us make the turning point a parameter. PyMC3 did not allow booleans from parameters, so we had to manually approximate this time value.

<center><img src="plots/OneBandPyMc3.png" alt="drawing" width="900"/></center>

*Melanie*
## Part IV: Hierarchical GPs in PyMC3
For $N$ bands, we generate six random variables for our quadratic parameters: 
* $N$-dimensional Gaussian distributions centered on polyfit values
$$ m_1 = a_1 t^2 + b_1 t + c_1 $$
$$ m_2 = a_2 t^2 + b_2 t + c_2 $$
* $N$ gps with same mean model, squared exponential kernel
$$ k(x, x') = \text{exp} \left[ - \frac{(x-x')^2}{2 \ell^2} \right] $$
* Condition on low-error values

The usefulness of a hierarchical model is that our parameters for each band could be drawn from the same distribution, so we can model the similarities in the population but allow individual parameter estimates. Doing so *shrinks* the variance between the parameter estimates.

Other parameters for consideration were the smoothing length, noise, and the "turn over" time for the spline. After tweaking, we did end up drawing the kernel smoothing length from a distribution as well.

<center><img src="plots/AllBandPyMc3.png" alt="drawing" width="900"/></center>

*Alex*
## Part V: Converting Posteriors to Bolometric Lightcurves

* First, we took the median of all posteriors in each band.
* We then estimated model error by adding the standard deviation of the posteriors in quadrature across the bands.
* We used trapezoidal integration to get the bolometric flux, and from that, luminosity.

To convert from bolometric flux to bolometric luminosity, we use the following relationship: 

$$ L_{bol} = F_{bol}  4 \pi d_{SN}^2$$

Where $d_{SN}$ is the distance to the supernova.

<center><img src="plots/Bolometric.png" alt="drawing" width="900"/></center>

*Alex*
## Part VIa: Estimating Temperature Evolution with Stefan-Boltzmann

If we assume a blackbody, then we can get a (ballpark) estimate for the photospheric temperature of the supernova over time with the equation:

$$
L_{bol}(MJD) = 4 \pi R(MJD)^2 \sigma_{SB} T_{eff}(MJD)^4
$$

We have decided as a start to assume a fixed photospheric radius. This also assumes our calculated $L_{bol}$ captures the full SN emission. 

<center><img src="plots/SB_curves.png" alt="drawing" width="900"/></center>

How can we do better? 

*Alex*
## Part VIb: Obtaining Temperature from a Blackbody Fit
In reality, trapezoidal integration over observed bands only captures a fraction of the full SED. 

<center><img src="./SED1.png" alt="drawing" width="400"/></center>


(ASAS-SN)

Other people have made packages that estimate bolometric lightcurves (SuperBoL and superbol, for example). In superbol, they fit a blackbody curve to the observed flux at each time to derive temperature and radius. We attempted to do the same.

<center><img src="plots/BB_curves.png" alt="drawing" width="900"/></center>

*The Audience*
## Summary

Our goal was to get **temperatures** from supernova data using **Gaussian Processes** in a **hierarchical model**. To do so, we had to consult literature and our own astrophysical knowledge about what model parameters might make sense. We've discovered: 
* Tradeoff between data fitting and model fitting
* Error propagation is hard

But, in the end, we got temperatures. Ta-da!